In [104]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 12: The N-Body Problem ---</h2><p>The space near Jupiter is not a very safe place; you need to be careful of a <a href="https://en.wikipedia.org/wiki/Great_Red_Spot">big distracting red spot</a>, extreme <a href="https://en.wikipedia.org/wiki/Magnetosphere_of_Jupiter">radiation</a>, and a <a href="https://en.wikipedia.org/wiki/Moons_of_Jupiter#List">whole lot of moons</a> swirling around.  You decide to start by tracking the four largest moons: <em>Io</em>, <em>Europa</em>, <em>Ganymede</em>, and <em>Callisto</em>.</p>
<p>After a brief scan, you calculate the <em>position of each moon</em> (your puzzle input). You just need to <em>simulate their motion</em> so you can <span title="Or you could just leave, but where's the fun in that?">avoid them</span>.</p>
<p>Each moon has a 3-dimensional position (<code>x</code>, <code>y</code>, and <code>z</code>) and a 3-dimensional velocity.  The position of each moon is given in your scan; the <code>x</code>, <code>y</code>, and <code>z</code> velocity of each moon starts at <code>0</code>.</p>
<p>Simulate the motion of the moons in <em>time steps</em>. Within each time step, first update the velocity of every moon by applying <em>gravity</em>. Then, once all moons' velocities have been updated, update the position of every moon by applying <em>velocity</em>. Time progresses by one step once all of the positions are updated.</p>
<p>To apply <em>gravity</em>, consider every <em>pair</em> of moons. On each axis (<code>x</code>, <code>y</code>, and <code>z</code>), the velocity of each moon changes by <em>exactly +1 or -1</em> to pull the moons together.  For example, if Ganymede has an <code>x</code> position of <code>3</code>, and Callisto has a <code>x</code> position of <code>5</code>, then Ganymede's <code>x</code> velocity <em>changes by +1</em> (because <code>5 &gt; 3</code>) and Callisto's <code>x</code> velocity <em>changes by -1</em> (because <code>3 &lt; 5</code>). However, if the positions on a given axis are the same, the velocity on that axis <em>does not change</em> for that pair of moons.</p>
<p>Once all gravity has been applied, apply <em>velocity</em>: simply add the velocity of each moon to its own position. For example, if Europa has a position of <code>x=1, y=2, z=3</code> and a velocity of <code>x=-2, y=0,z=3</code>, then its new position would be <code>x=-1, y=2, z=6</code>. This process does not modify the velocity of any moon.</p>
<p>For example, suppose your scan reveals the following positions:</p>
<pre><code>&lt;x=-1, y=0, z=2&gt;
&lt;x=2, y=-10, z=-7&gt;
&lt;x=4, y=-8, z=8&gt;
&lt;x=3, y=5, z=-1&gt;
</code></pre>
<p>Simulating the motion of these moons would produce the following:</p>
<pre><code>After 0 steps:
pos=&lt;x=-1, y=  0, z= 2&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;
pos=&lt;x= 2, y=-10, z=-7&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;
pos=&lt;x= 4, y= -8, z= 8&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;
pos=&lt;x= 3, y=  5, z=-1&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;

After 1 step:
pos=&lt;x= 2, y=-1, z= 1&gt;, vel=&lt;x= 3, y=-1, z=-1&gt;
pos=&lt;x= 3, y=-7, z=-4&gt;, vel=&lt;x= 1, y= 3, z= 3&gt;
pos=&lt;x= 1, y=-7, z= 5&gt;, vel=&lt;x=-3, y= 1, z=-3&gt;
pos=&lt;x= 2, y= 2, z= 0&gt;, vel=&lt;x=-1, y=-3, z= 1&gt;

After 2 steps:
pos=&lt;x= 5, y=-3, z=-1&gt;, vel=&lt;x= 3, y=-2, z=-2&gt;
pos=&lt;x= 1, y=-2, z= 2&gt;, vel=&lt;x=-2, y= 5, z= 6&gt;
pos=&lt;x= 1, y=-4, z=-1&gt;, vel=&lt;x= 0, y= 3, z=-6&gt;
pos=&lt;x= 1, y=-4, z= 2&gt;, vel=&lt;x=-1, y=-6, z= 2&gt;

After 3 steps:
pos=&lt;x= 5, y=-6, z=-1&gt;, vel=&lt;x= 0, y=-3, z= 0&gt;
pos=&lt;x= 0, y= 0, z= 6&gt;, vel=&lt;x=-1, y= 2, z= 4&gt;
pos=&lt;x= 2, y= 1, z=-5&gt;, vel=&lt;x= 1, y= 5, z=-4&gt;
pos=&lt;x= 1, y=-8, z= 2&gt;, vel=&lt;x= 0, y=-4, z= 0&gt;

After 4 steps:
pos=&lt;x= 2, y=-8, z= 0&gt;, vel=&lt;x=-3, y=-2, z= 1&gt;
pos=&lt;x= 2, y= 1, z= 7&gt;, vel=&lt;x= 2, y= 1, z= 1&gt;
pos=&lt;x= 2, y= 3, z=-6&gt;, vel=&lt;x= 0, y= 2, z=-1&gt;
pos=&lt;x= 2, y=-9, z= 1&gt;, vel=&lt;x= 1, y=-1, z=-1&gt;

After 5 steps:
pos=&lt;x=-1, y=-9, z= 2&gt;, vel=&lt;x=-3, y=-1, z= 2&gt;
pos=&lt;x= 4, y= 1, z= 5&gt;, vel=&lt;x= 2, y= 0, z=-2&gt;
pos=&lt;x= 2, y= 2, z=-4&gt;, vel=&lt;x= 0, y=-1, z= 2&gt;
pos=&lt;x= 3, y=-7, z=-1&gt;, vel=&lt;x= 1, y= 2, z=-2&gt;

After 6 steps:
pos=&lt;x=-1, y=-7, z= 3&gt;, vel=&lt;x= 0, y= 2, z= 1&gt;
pos=&lt;x= 3, y= 0, z= 0&gt;, vel=&lt;x=-1, y=-1, z=-5&gt;
pos=&lt;x= 3, y=-2, z= 1&gt;, vel=&lt;x= 1, y=-4, z= 5&gt;
pos=&lt;x= 3, y=-4, z=-2&gt;, vel=&lt;x= 0, y= 3, z=-1&gt;

After 7 steps:
pos=&lt;x= 2, y=-2, z= 1&gt;, vel=&lt;x= 3, y= 5, z=-2&gt;
pos=&lt;x= 1, y=-4, z=-4&gt;, vel=&lt;x=-2, y=-4, z=-4&gt;
pos=&lt;x= 3, y=-7, z= 5&gt;, vel=&lt;x= 0, y=-5, z= 4&gt;
pos=&lt;x= 2, y= 0, z= 0&gt;, vel=&lt;x=-1, y= 4, z= 2&gt;

After 8 steps:
pos=&lt;x= 5, y= 2, z=-2&gt;, vel=&lt;x= 3, y= 4, z=-3&gt;
pos=&lt;x= 2, y=-7, z=-5&gt;, vel=&lt;x= 1, y=-3, z=-1&gt;
pos=&lt;x= 0, y=-9, z= 6&gt;, vel=&lt;x=-3, y=-2, z= 1&gt;
pos=&lt;x= 1, y= 1, z= 3&gt;, vel=&lt;x=-1, y= 1, z= 3&gt;

After 9 steps:
pos=&lt;x= 5, y= 3, z=-4&gt;, vel=&lt;x= 0, y= 1, z=-2&gt;
pos=&lt;x= 2, y=-9, z=-3&gt;, vel=&lt;x= 0, y=-2, z= 2&gt;
pos=&lt;x= 0, y=-8, z= 4&gt;, vel=&lt;x= 0, y= 1, z=-2&gt;
pos=&lt;x= 1, y= 1, z= 5&gt;, vel=&lt;x= 0, y= 0, z= 2&gt;

After 10 steps:
pos=&lt;x= 2, y= 1, z=-3&gt;, vel=&lt;x=-3, y=-2, z= 1&gt;
pos=&lt;x= 1, y=-8, z= 0&gt;, vel=&lt;x=-1, y= 1, z= 3&gt;
pos=&lt;x= 3, y=-6, z= 1&gt;, vel=&lt;x= 3, y= 2, z=-3&gt;
pos=&lt;x= 2, y= 0, z= 4&gt;, vel=&lt;x= 1, y=-1, z=-1&gt;
</code></pre>

<p>Then, it might help to calculate the <em>total energy in the system</em>. The total energy for a single moon is its <em>potential energy</em> multiplied by its <em>kinetic energy</em>. A moon's <em>potential energy</em> is the sum of the <a href="https://en.wikipedia.org/wiki/Absolute_value">absolute values</a> of its <code>x</code>, <code>y</code>, and <code>z</code> position coordinates. A moon's <em>kinetic energy</em> is the sum of the absolute values of its velocity coordinates.  Below, each line shows the calculations for a moon's potential energy (<code>pot</code>), kinetic energy (<code>kin</code>), and total energy:</p>
<pre><code>Energy after 10 steps:
pot: 2 + 1 + 3 =  6;   kin: 3 + 2 + 1 = 6;   total:  6 * 6 = 36
pot: 1 + 8 + 0 =  9;   kin: 1 + 1 + 3 = 5;   total:  9 * 5 = 45
pot: 3 + 6 + 1 = 10;   kin: 3 + 2 + 3 = 8;   total: 10 * 8 = 80
pot: 2 + 0 + 4 =  6;   kin: 1 + 1 + 1 = 3;   total:  6 * 3 = 18
Sum of total energy: 36 + 45 + 80 + 18 = <em>179</em>
</code></pre>
<p>In the above example, adding together the total energy for all moons after 10 steps produces the total energy in the system, <code><em>179</em></code>.</p>
<p>Here's a second example:</p>
<pre><code>&lt;x=-8, y=-10, z=0&gt;
&lt;x=5, y=5, z=10&gt;
&lt;x=2, y=-7, z=3&gt;
&lt;x=9, y=-8, z=-3&gt;
</code></pre>
<p>Every ten steps of simulation for 100 steps produces:</p>
<pre><code>After 0 steps:
pos=&lt;x= -8, y=-10, z=  0&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;
pos=&lt;x=  5, y=  5, z= 10&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;
pos=&lt;x=  2, y= -7, z=  3&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;
pos=&lt;x=  9, y= -8, z= -3&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;

After 10 steps:
pos=&lt;x= -9, y=-10, z= 1&gt;, vel=&lt;x= -2, y= -2, z= -1&gt;
pos=&lt;x= 4, y= 10, z= 9&gt;, vel=&lt;x= -3, y= 7, z= -2&gt;
pos=&lt;x= 8, y=-10, z= -3&gt;, vel=&lt;x= 5, y= -1, z= -2&gt;
pos=&lt;x= 5, y=-10, z= 3&gt;, vel=&lt;x= 0, y= -4, z= 5&gt;

After 20 steps:
pos=&lt;x=-10, y= 3, z= -4&gt;, vel=&lt;x= -5, y= 2, z= 0&gt;
pos=&lt;x= 5, y=-25, z= 6&gt;, vel=&lt;x= 1, y= 1, z= -4&gt;
pos=&lt;x= 13, y= 1, z= 1&gt;, vel=&lt;x= 5, y= -2, z= 2&gt;
pos=&lt;x= 0, y= 1, z= 7&gt;, vel=&lt;x= -1, y= -1, z= 2&gt;

After 30 steps:
pos=&lt;x= 15, y= -6, z= -9&gt;, vel=&lt;x= -5, y= 4, z= 0&gt;
pos=&lt;x= -4, y=-11, z= 3&gt;, vel=&lt;x= -3, y=-10, z= 0&gt;
pos=&lt;x= 0, y= -1, z= 11&gt;, vel=&lt;x= 7, y= 4, z= 3&gt;
pos=&lt;x= -3, y= -2, z= 5&gt;, vel=&lt;x= 1, y= 2, z= -3&gt;

After 40 steps:
pos=&lt;x= 14, y=-12, z= -4&gt;, vel=&lt;x= 11, y= 3, z= 0&gt;
pos=&lt;x= -1, y= 18, z= 8&gt;, vel=&lt;x= -5, y= 2, z= 3&gt;
pos=&lt;x= -5, y=-14, z= 8&gt;, vel=&lt;x= 1, y= -2, z= 0&gt;
pos=&lt;x= 0, y=-12, z= -2&gt;, vel=&lt;x= -7, y= -3, z= -3&gt;

After 50 steps:
pos=&lt;x=-23, y= 4, z= 1&gt;, vel=&lt;x= -7, y= -1, z= 2&gt;
pos=&lt;x= 20, y=-31, z= 13&gt;, vel=&lt;x= 5, y= 3, z= 4&gt;
pos=&lt;x= -4, y= 6, z= 1&gt;, vel=&lt;x= -1, y= 1, z= -3&gt;
pos=&lt;x= 15, y= 1, z= -5&gt;, vel=&lt;x= 3, y= -3, z= -3&gt;

After 60 steps:
pos=&lt;x= 36, y=-10, z= 6&gt;, vel=&lt;x= 5, y= 0, z= 3&gt;
pos=&lt;x=-18, y= 10, z= 9&gt;, vel=&lt;x= -3, y= -7, z= 5&gt;
pos=&lt;x= 8, y=-12, z= -3&gt;, vel=&lt;x= -2, y= 1, z= -7&gt;
pos=&lt;x=-18, y= -8, z= -2&gt;, vel=&lt;x= 0, y= 6, z= -1&gt;

After 70 steps:
pos=&lt;x=-33, y= -6, z= 5&gt;, vel=&lt;x= -5, y= -4, z= 7&gt;
pos=&lt;x= 13, y= -9, z= 2&gt;, vel=&lt;x= -2, y= 11, z= 3&gt;
pos=&lt;x= 11, y= -8, z= 2&gt;, vel=&lt;x= 8, y= -6, z= -7&gt;
pos=&lt;x= 17, y= 3, z= 1&gt;, vel=&lt;x= -1, y= -1, z= -3&gt;

After 80 steps:
pos=&lt;x= 30, y= -8, z= 3&gt;, vel=&lt;x= 3, y= 3, z= 0&gt;
pos=&lt;x= -2, y= -4, z= 0&gt;, vel=&lt;x= 4, y=-13, z= 2&gt;
pos=&lt;x=-18, y= -7, z= 15&gt;, vel=&lt;x= -8, y= 2, z= -2&gt;
pos=&lt;x= -2, y= -1, z= -8&gt;, vel=&lt;x= 1, y= 8, z= 0&gt;

After 90 steps:
pos=&lt;x=-25, y= -1, z= 4&gt;, vel=&lt;x= 1, y= -3, z= 4&gt;
pos=&lt;x= 2, y= -9, z= 0&gt;, vel=&lt;x= -3, y= 13, z= -1&gt;
pos=&lt;x= 32, y= -8, z= 14&gt;, vel=&lt;x= 5, y= -4, z= 6&gt;
pos=&lt;x= -1, y= -2, z= -8&gt;, vel=&lt;x= -3, y= -6, z= -9&gt;

After 100 steps:
pos=&lt;x= 8, y=-12, z= -9&gt;, vel=&lt;x= -7, y= 3, z= 0&gt;
pos=&lt;x= 13, y= 16, z= -3&gt;, vel=&lt;x= 3, y=-11, z= -5&gt;
pos=&lt;x=-29, y=-11, z= -1&gt;, vel=&lt;x= -3, y= 7, z= 4&gt;
pos=&lt;x= 16, y=-13, z= 23&gt;, vel=&lt;x= 7, y= 1, z= 1&gt;

Energy after 100 steps:
pot: 8 + 12 + 9 = 29; kin: 7 + 3 + 0 = 10; total: 29 _ 10 = 290
pot: 13 + 16 + 3 = 32; kin: 3 + 11 + 5 = 19; total: 32 _ 19 = 608
pot: 29 + 11 + 1 = 41; kin: 3 + 7 + 4 = 14; total: 41 _ 14 = 574
pot: 16 + 13 + 23 = 52; kin: 7 + 1 + 1 = 9; total: 52 _ 9 = 468
Sum of total energy: 290 + 608 + 574 + 468 = <em>1940</em>
</code></pre>

<p><em>What is the total energy in the system</em> after simulating the moons given in your scan for <code>1000</code> steps?</p>
</article>


In [105]:
from math import gcd, lcm, prod
from re import findall

from more_itertools import distinct_combinations
from sympy import primefactors

type MoonState = tuple[int, int, int, int, int, int]


class Moon:

    integer_width_for_repr = 4

    def __init__(
        self, x: int, y: int, z: int, dx: int = 0, dy: int = 0, dz: int = 0
    ) -> None:
        self.x, self.y, self.z = x, y, z
        self.dx, self.dy, self.dz = dx, dy, dz
        self.history = {}

    def accelerate(self, other: Moon) -> None:
        self.dx += self.accelerate_dimension(self.x, other.x)
        self.dy += self.accelerate_dimension(self.y, other.y)
        self.dz += self.accelerate_dimension(self.z, other.z)

    def move(self) -> None:
        self.x += self.dx
        self.y += self.dy
        self.z += self.dz

    def state(self) -> MoonState:
        return self.x, self.y, self.z, self.dx, self.dy, self.dz

    @property
    def pot(self) -> int:
        return abs(self.x) + abs(self.y) + abs(self.z)

    @property
    def kin(self) -> int:
        return abs(self.dx) + abs(self.dy) + abs(self.dz)

    @classmethod
    def accelerate_dimension(cls, own_value: int, other_value: int) -> int:
        if own_value < other_value:
            return 1
        if own_value > other_value:
            return -1
        return 0

    def __repr__(self) -> str:
        w = self.integer_width_for_repr
        return f"pos=<x={self.x:>{w}}, y={self.y:>{w}}, z={self.z:>{w}}>, vel=<x={self.dx:>{w}}, y={self.dy:>{w}}, z={self.dz:>{w}}>"


class History:
    def __init__(self, value: tuple[int, ...]) -> None:
        self.history = value
        self.cycle_length = -1

    def has_cycle(self) -> bool:
        return self.cycle_length != -1

    def add(self, value: tuple[int, ...], step: int) -> None:
        if not self.has_cycle() and value == self.history:
            self.cycle_length = step + 1


class SaturnHistory:
    def __init__(
        self, xs: tuple[int, ...], ys: tuple[int, ...], zs: tuple[int, ...]
    ) -> None:
        self.x_history = History(xs)
        self.y_history = History(ys)
        self.z_history = History(zs)

    def all_have_cycles(self) -> bool:
        return (
            self.x_history.has_cycle()
            and self.y_history.has_cycle()
            and self.z_history.has_cycle()
        )

    def add(
        self, xs: tuple[int, ...], ys: tuple[int, ...], zs: tuple[int, ...], step: int
    ) -> None:
        if self.all_have_cycles():
            return
        self.x_history.add(xs, step)
        self.y_history.add(ys, step)
        self.z_history.add(zs, step)

    def lcm(self) -> int:
        return lcm(
            self.x_history.cycle_length,
            self.y_history.cycle_length,
            self.z_history.cycle_length,
        )


class Saturn:
    def __init__(self, s: str) -> None:
        self.moons = [
            Moon(*[int(i) for i in findall(r"-?\d+", l)])
            for l in s.strip().splitlines()
        ]

    def apply_gravity(self) -> None:
        for mi, mj in distinct_combinations(self.moons, 2):
            if mi is not mj:
                mj.accelerate(mi)
                mi.accelerate(mj)

    def apply_velocity(self) -> None:
        for m in self.moons:
            m.move()

    def simulating(
        self, steps: int, do_print: bool = False, print_every_nth_step: int = 1
    ) -> int:
        if do_print:
            print(f"After 0 steps:")
            print(self)
            print()

        for step in range(1, steps + 1):
            self.apply_gravity()
            self.apply_velocity()
            if do_print and step % print_every_nth_step == 0:
                print(f"After {step} steps:")
                print(self)
                print()

        return self.total_energy()

    def state(self) -> tuple[MoonState, ...]:
        return tuple(m.state() for m in self.moons)

    def steps_state_seen_before(self) -> int:

        step = 0
        xs = tuple(m.x for m in self.moons)
        ys = tuple(m.y for m in self.moons)
        zs = tuple(m.z for m in self.moons)
        sh = SaturnHistory(xs, ys, zs)

        while not sh.all_have_cycles():
            step += 1
            self.apply_gravity()
            self.apply_velocity()

            xs = tuple(m.x for m in self.moons)
            ys = tuple(m.y for m in self.moons)
            zs = tuple(m.z for m in self.moons)
            sh.add(xs, ys, zs, step)

        return sh.lcm()

    def steps_state_seen_before_bf(self) -> int:
        current_state = self.state()
        states_seen = {current_state}

        steps = 0

        while True:
            self.apply_gravity()
            self.apply_velocity()
            current_state = self.state()
            steps += 1
            if current_state in states_seen:
                return steps
            states_seen.add(current_state)

    def total_energy(self) -> int:
        return sum(m.pot * m.kin for m in self.moons)

    def __repr__(self) -> str:
        return "\n".join(map(str, self.moons))


example_1 = """
<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>
"""

example_2 = """
<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>
"""

Moon.integer_width_for_repr = 3

# print()
# Saturn(example_1).simulating(10, True, 1)
# print()
# Saturn(example_2).simulating(100, True, 10)

print(f"Example 1:{Saturn(example_1).simulating(10, False)} should be 179")
print(f"Example 2:{Saturn(example_2).simulating(100, False)} should be 1940")

Example 1:179 should be 179
Example 2:1940 should be 1940


In [106]:
puzzle = """
<x=-6, y=2, z=-9>
<x=12, y=-14, z=-4>
<x=9, y=5, z=-6>
<x=-1, y=-4, z=9>
"""

print(f"Part 1:{Saturn(puzzle).simulating(1_000, False)}")

Part 1:14907


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>14907</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>All this drifting around in space makes you wonder about the nature of the universe.  Does history really repeat itself?  You're curious whether the moons will ever return to a previous state.</p>
<p>Determine <em>the number of steps</em> that must occur before all of the moons' <em>positions and velocities</em> exactly match a previous point in time.</p>
<p>For example, the first example above takes <code>2772</code> steps before they exactly match a previous point in time; it eventually returns to the initial state:</p>
<pre><code>After 0 steps:
pos=&lt;x= -1, y=  0, z=  2&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;
pos=&lt;x=  2, y=-10, z= -7&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;
pos=&lt;x=  4, y= -8, z=  8&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;
pos=&lt;x=  3, y=  5, z= -1&gt;, vel=&lt;x=  0, y=  0, z=  0&gt;

After 2770 steps:
pos=&lt;x= 2, y= -1, z= 1&gt;, vel=&lt;x= -3, y= 2, z= 2&gt;
pos=&lt;x= 3, y= -7, z= -4&gt;, vel=&lt;x= 2, y= -5, z= -6&gt;
pos=&lt;x= 1, y= -7, z= 5&gt;, vel=&lt;x= 0, y= -3, z= 6&gt;
pos=&lt;x= 2, y= 2, z= 0&gt;, vel=&lt;x= 1, y= 6, z= -2&gt;

After 2771 steps:
pos=&lt;x= -1, y= 0, z= 2&gt;, vel=&lt;x= -3, y= 1, z= 1&gt;
pos=&lt;x= 2, y=-10, z= -7&gt;, vel=&lt;x= -1, y= -3, z= -3&gt;
pos=&lt;x= 4, y= -8, z= 8&gt;, vel=&lt;x= 3, y= -1, z= 3&gt;
pos=&lt;x= 3, y= 5, z= -1&gt;, vel=&lt;x= 1, y= 3, z= -1&gt;

After 2772 steps:
pos=&lt;x= -1, y= 0, z= 2&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;
pos=&lt;x= 2, y=-10, z= -7&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;
pos=&lt;x= 4, y= -8, z= 8&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;
pos=&lt;x= 3, y= 5, z= -1&gt;, vel=&lt;x= 0, y= 0, z= 0&gt;
</code></pre>

<p>Of course, the universe might last for a <em>very long time</em> before repeating.  Here's a copy of the second example from above:</p>
<pre><code>&lt;x=-8, y=-10, z=0&gt;
&lt;x=5, y=5, z=10&gt;
&lt;x=2, y=-7, z=3&gt;
&lt;x=9, y=-8, z=-3&gt;
</code></pre>
<p>This set of initial positions takes <code>4686774924</code> steps before it repeats a previous state! Clearly, you might need to <em>find a more efficient way to simulate the universe</em>.</p>
<p><em>How many steps does it take</em> to reach the first state that exactly matches a previous state?</p>
</article>

</main>


In [107]:
print(
    f"Example 1 Bruteforce: {Saturn(example_1).steps_state_seen_before_bf()} should be 2772"
)
print(f"Example 1: {Saturn(example_1).steps_state_seen_before()} should be 2772")
print()
print(f"Example 2: {Saturn(example_2).steps_state_seen_before()} should be 4686774924")

Example 1 Bruteforce: 2772 should be 2772
Example 1: 2772 should be 2772

Example 2: 4686774924 should be 4686774924


In [108]:
print(f"Part II: {Saturn(puzzle).steps_state_seen_before()}")

Part II: 467081194429464


<link href="style.css" rel="stylesheet"></link>
<main class="read-aloud">

<p>Your puzzle answer was <code>467081194429464</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
